In [2]:
import csv
import data_processing
import algorithm
from re import findall
import recipe_parser

In [3]:
with open('data/recipes.csv', 'r') as f:
    recipes = list(csv.DictReader(f))

In [4]:
recipes = recipes[:10000]
inv_idx = data_processing.build_inverted_index(recipes)
id_to_recipe = data_processing.build_id_to_recipe(recipes)
idf = data_processing.build_idf(inv_idx, len(recipes))
recipe_norms = data_processing.build_recipe_norms(inv_idx, idf)

In [6]:
res = algorithm.cosine_similarity("lemon dessert", inv_idx, idf, recipe_norms)

{'protein': 2.3415371388258905, 'fat': 5.0397848661058635, 'weeknight': 2.154570167148848, 'freezer': 6.310432456049533, 'granulated': 5.5328248773859805, 'juice': 2.9224835302979044, 'easy': 1.341171575656346, 'summer': 4.733123527871812, 'healthy': 2.4065984188743528, 'low': 1.7933567760166045, 'berry': 7.965784284662087, 'yogurt': 5.435963338133392, 'vanilla': 2.9567955014348324, 'blue': 7.480357457491845, 'blueberries': 7.1791879227712805, 'sugar': 1.3455644591546008, 'of': 3.437525541903675, 'frozen': 4.828280760912152, 'lemon': 2.935668953754016, 'free': 4.268121651191569, 'desserts': 6.844768883700721, 'dessert': 2.0715728231243147, 'cholesterol': 2.519528054772523, 'salt': 1.2337864980183448, 'peppercorns': 6.764150423492437, 'poppy': 7.861447624847352, 'stove': 3.3393451479647718, 'poultry': 3.301870442546109, 'leaf': 4.9976935326168315, 'hot': 5.153286059328523, 'cardamom': 6.687799537362322, 'clove': 4.925768605814208, 'chicken': 3.0880400347130847, 'saffron': 7.702749878828

In [7]:
# Print out the top 10 recipe names
for recipe_id, score in res[:10]:
    print(id_to_recipe[recipe_id]['name'], score)

Rice Pudding 0.12546715457826013
Lemon Poppy Seed Cake 0.11975219083741352
Green Tomato Pie 0.11440767944197215
Lemon Cake 0.11337507196141429
Vinegar Pie 0.1126777024986445
Lemon Sour Cream Pie 0.11215789237072495
Lemon Sponge Pie 0.1100246145192226
Orange Marmalade 0.10999133807435493
Ricotta Cheesecake 0.10658745728313504
Lemon-Sour Cream Pie 0.10578394745474305


In [15]:
res2 = algorithm.algorithm(["ice cream", "strawberry", "frozen yogurt"], inv_idx, idf, recipe_norms)

In [16]:
# Print out the top 10 recipe names
for recipe_id, score in res2[:10]:
    print(id_to_recipe[recipe_id]['name'])

Ice Cream
Strawberry Milk
Homemade Frozen Yogurt
Lemon Dessert
Strawberry Vinegar
Chocolate Frozen Yogurt
Cinnamon Ice Cream
Strawberry Cake
Frozen Fruit and Yogurt Pops
Roasted Garlic Ice Cream
